# Data science on the HMRC skilled worker data set
The below csv file has a list of companies capable of skilled worker sponsornship in the UK

In [27]:
# importing the libraries
import numpy as np
import pandas as pd

In [28]:
import os
# Print the current python enviornments base folder, as depending on the python env you setup the relative folder to the csv below should be adjusted
print(os.getcwd())
skilled_worker_data_2025_06_19 = pd.read_csv('./2025-06-19-Worker.csv')
skilled_worker_data_current = pd.read_csv('./2025-07-03-Worker.csv')
print(skilled_worker_data_2025_06_19.count())
print(skilled_worker_data_current.count())
# comparing the current data to an older data
difference = skilled_worker_data_current.count() - skilled_worker_data_2025_06_19.count()
organisations_count_diff = difference['Organisation Name']

print(f"""\nWe see how that {organisations_count_diff} companies 
have been {"added" if organisations_count_diff >= 0 else "removed"} to the list.\n""")
print(difference)

/Users/nikilkuruvilla/projects/python-ml
Organisation Name    133028
Town/City            133025
County                45163
Type & Rating        133028
Route                133028
dtype: int64
Organisation Name    133798
Town/City            133795
County                45423
Type & Rating        133798
Route                133798
dtype: int64

We see how that 770 companies 
have been added to the list.

Organisation Name    770
Town/City            770
County               260
Type & Rating        770
Route                770
dtype: int64


### shape of the data

In [29]:
skilled_worker_data_current.shape

(133798, 5)

### info

In [30]:
skilled_worker_data_current.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133798 entries, 0 to 133797
Data columns (total 5 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   Organisation Name  133798 non-null  object
 1   Town/City          133795 non-null  object
 2   County             45423 non-null   object
 3   Type & Rating      133798 non-null  object
 4   Route              133798 non-null  object
dtypes: object(5)
memory usage: 5.1+ MB


### head / tail

In [31]:
skilled_worker_data_current.head()

,Organisation Name,Town/City,County,Type & Rating,Route
0,McMullan Shellfish,Ballymena,Co Antrim,Worker (A rating),Skilled Worker
1,(A1F1 Limited T/A ) Ultrasound Direct London,Croydon,London,Worker (A rating),Skilled Worker
2,(IECC Care) Independent Excel Care Consortium ...,Colchester,NaN,Worker (A rating),Skilled Worker
3,*ABOUTCARE HASTINGS LTD,East Sussex,NaN,Worker (A rating),Skilled Worker
4,.LITTLE NOORIYAH LTD,Smethwick,NaN,Worker (A rating),Skilled Worker


In [32]:
skilled_worker_data_current.tail()

,Organisation Name,Town/City,County,Type & Rating,Route
133793,ZZA CONSULTING LIMITED,LONDON,NaN,Worker (A rating),Skilled Worker
133794,ZZIY Ltd,High Wycombe,NaN,Worker (A rating),Skilled Worker
133795,ZZN STUDIO LTD,HAMPTON,NaN,Worker (A rating),Skilled Worker
133796,Zzoomm Plc,Oxford,NaN,Worker (A rating),Skilled Worker
133797,ZZZ Limited,London,NaN,Worker (A rating),Skilled Worker


### unique

In [33]:
skilled_worker_data_current['County'].unique()

array(['Co Antrim', 'London', nan, ..., 'Ascot', 'Gillingham',
       'Wes Yorkshire'], dtype=object)

### value_counts

In [34]:
skilled_worker_data_current['Town/City'].value_counts()

Town/City
London             37214
LONDON              3327
Birmingham          2829
Manchester          2628
Bristol             1271
                   ...  
Burton on trent        1
Lees                   1
Lytham st annes        1
Roehampton             1
Northwood hills        1
Name: count, Length: 7964, dtype: int64

### value_counts(normalize=True)

The below data tells me that the number of companies with Town/City having `London` makes up `27%` of the total companies available in the skilled worker category in UK.

`LONDON` contributes to `2.4%`
`Birmingham`-`2.1%` and so forth..

In [35]:
skilled_worker_data_current['Town/City'].value_counts(normalize=True)

Town/City
London             0.278142
LONDON             0.024866
Birmingham         0.021144
Manchester         0.019642
Bristol            0.009500
                     ...   
Burton on trent    0.000007
Lees               0.000007
Lytham st annes    0.000007
Roehampton         0.000007
Northwood hills    0.000007
Name: proportion, Length: 7964, dtype: float64

In [36]:
current_highest_contributors = skilled_worker_data_current['Town/City'].value_counts(normalize=True)
previous_highest_contributors = skilled_worker_data_2025_06_19['Town/City'].value_counts(normalize=True)
diff_highest_contributors = (current_highest_contributors - previous_highest_contributors)*100
# write to excel
diff_highest_contributors.to_excel('./diff.xlsx')
print(diff_highest_contributors)

Town/City
 Aberdeen        -0.000009
 Aberdeenshire   -0.000004
 Abingdon        -0.000004
 Accrington      -0.000009
 Acton           -0.000004
                    ...   
worksop          -0.000004
wormley          -0.000004
worthing         -0.000004
yattendon        -0.000004
york             -0.000017
Name: proportion, Length: 7968, dtype: float64


### mode()

The mode on a column returns back the most occuring value.

Hence data['column'].mode() returns an array of the most occuring values in a list. so if 'London' and another city 'Xyz' took up the most values it would return both, otherwise just the one thats the most repeating. You then use the index
to access the value as seen below.

In [37]:
skilled_worker_data_current['Town/City'].mode()[0]

'London'

In [38]:
skilled_worker_data_current['Type & Rating'].mode()[0]

'Worker (A rating)'

In [39]:
skilled_worker_data_current['Route'].mode()[0]

'Skilled Worker'

Interesting how `Surrey` is the most repeated `County` in the HMRC data set.

What it means is that most of the companies sponsoring are in the Surrey county according to the data set.
But this could also be because that information is not fully captured with all the companies

In [40]:
skilled_worker_data_current['County'].mode()[0]

'Surrey'

# Fuzzy search on organisation names

### What is Levenshtein Distance?

The Levenshtein distance is a measure of the difference between two strings. It is defined as the minimum number of single-character edits (insertions, deletions, or substitutions) required to change one string into the other.

For example, the Levenshtein distance between "kitten" and "sitting" is 3, since the following three edits change one into the other, and there is no way to do it with fewer than three edits:

1.  **k**itten → **s**itten (substitution of "s" for "k")
2.  sitt**e**n → sitt**i**n (substitution of "i" for "e")
3.  sittin → sittin**g** (insertion of "g" at the end)

The `fuzzywuzzy` library uses the Levenshtein distance to calculate the similarity ratios between strings. The `python-Levenshtein` library is a C implementation of the algorithm, which makes the calculations much faster than if they were done in pure Python. This is why we installed it alongside `fuzzywuzzy`.

In [41]:
# print(skilled_worker_data.columns)
# !pip install -q fuzzywuzzy python-Levenshtein
from fuzzywuzzy import fuzz

company_name = input("Enter a company name")

def get_fuzzy_score(company) -> int:
    return fuzz.token_set_ratio(company_name.lower(), str(company).lower())

skilled_worker_data_current['fuzzy_score'] = skilled_worker_data_current['Organisation Name'].apply(get_fuzzy_score)

# Set a threshold for what is considered a "close match"
threshold = 80
matching_rows = skilled_worker_data_current[skilled_worker_data_current['fuzzy_score'] >= threshold]


if not matching_rows.empty:
    print(f"Yes, a close match to '{company_name}' exists. Showing matches with a score of {threshold} or higher:")
    print(matching_rows[['Organisation Name', 'Town/City', 'County', 'fuzzy_score']])
else:
    print(f"No close match to '{company_name}' found.")


Yes, a close match to 'masref' exists. Showing matches with a score of 80 or higher:
          Organisation Name Town/City County  fuzzy_score
73490  Masref Ltd t/a Nsave    London    NaN          100


### groupby()

In [42]:
# skilled_worker_data.groupby(['Town/City'])['County'].mode()[0]